<a href="https://colab.research.google.com/github/lab30041954/ML_IESE_Course/blob/main/%5BML-08%5D%20Classroom%20notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ML-08] Classroom notebook

### Creating a data folder

In [1]:
import os
os.mkdir('data/')

### Dowloading the zip files

In [2]:
import requests

In [3]:
gitpath = 'https://raw.githubusercontent.com/lab30041954/Data/main/'

In [4]:
gitlist = ['cats_train.zip', 'cats_test.zip', 'dogs_train.zip', 'dogs_test.zip']
for f in gitlist:
	r = requests.get(gitpath + f, stream=True)
	conn = open('data/' + f, mode='wb')
	conn.write(r.content)
	conn.close()

### Unzipping and removing the zip files

In [5]:
import zipfile

In [6]:
ziplist = [f for f in os.listdir('data/') if 'zip' in f]
for f in ziplist:
	zf = zipfile.ZipFile('data/' + f, 'r')
	zf.extractall('data/')
	del zf
	os.remove('data/' + f)

In [7]:
os.listdir('data/')

['dogs_train', 'cats_train', 'cats_test', 'dogs_test']

In [8]:
len(os.listdir('data/dogs_train/'))

1000

### Converting images to tensors

In [9]:
import numpy as np, matplotlib.pyplot as plt

In [10]:
def img_to_arr(f):
    arr = plt.imread(f)
    reshaped_arr = arr.reshape(1, 150, 150, 3)
    return reshaped_arr

## Training data

In [11]:
X_train = img_to_arr('data/dogs_train/' + os.listdir('data/dogs_train')[0])
X_train.shape

(1, 150, 150, 3)

In [12]:
for i in range(1, 1000):
    X_train = np.concatenate([X_train, img_to_arr('data/dogs_train/' + os.listdir('data/dogs_train')[i])])

In [13]:
for i in range(1000):
    X_train = np.concatenate([X_train, img_to_arr('data/cats_train/' + os.listdir('data/cats_train')[i])])

In [14]:
X_train = X_train/255

In [15]:
y_train = np.concatenate([np.ones(1000), np.zeros(1000)])

In [16]:
X_train.shape, y_train.shape

((2000, 150, 150, 3), (2000,))

### Test data

In [17]:
X_test = img_to_arr('data/dogs_test/' + os.listdir('data/dogs_test')[0])
for i in range(1, 500):
    X_test = np.concatenate([X_test, img_to_arr('data/dogs_test/' + os.listdir('data/dogs_test')[i])])
for i in range(500):
    X_test = np.concatenate([X_test, img_to_arr('data/cats_test/' + os.listdir('data/cats_test')[i])])
X_test = X_test/255
y_test = np.concatenate([np.ones(500), np.zeros(500)])
X_test.shape, y_test.shape

((1000, 150, 150, 3), (1000,))

### Training a CNN model from scratch

In [18]:
import os
os.environ['KERAS_BACKEND'] = 'jax'
from keras import Input, models, layers

/opt/anaconda3/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [19]:
input_tensor = Input(shape=(150, 150, 3))
x1 = layers.Conv2D(32, (3, 3), activation='relu')(input_tensor)
x2 = layers.MaxPooling2D((2, 2))(x1)
x3 = layers.Conv2D(64, (3, 3), activation='relu')(x2)
x4 = layers.MaxPooling2D((2, 2))(x3)
x5 = layers.Conv2D(128, (3, 3), activation='relu')(x4)
x6 = layers.MaxPooling2D((2, 2))(x5)
x7 = layers.Conv2D(128, (3, 3), activation='relu')(x6)
x8 = layers.MaxPooling2D((2, 2))(x7)
x9 = layers.Flatten()(x8)
x10 = layers.Dense(512, activation='relu')(x9)
output_tensor = layers.Dense(2, activation='softmax')(x10)
clf1 = models.Model(input_tensor, output_tensor)

In [20]:
clf1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,453,634 (13.17 MB)

 Trainable params: 3,453,634 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
clf1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
clf1.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test));

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 251ms/step - acc: 0.4955 - loss: 0.7188 - val_acc: 0.5000 - val_loss: 0.6930
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 15s 240ms/step - acc: 0.5565 - loss: 0.6788 - val_acc: 0.5780 - val_loss: 0.7019
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 247ms/step - acc: 0.6050 - loss: 0.6644 - val_acc: 0.5360 - val_loss: 0.7935
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 249ms/step - acc: 0.6335 - loss: 0.6363 - val_acc: 0.6230 - val_loss: 0.6358
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 251ms/step - acc: 0.6570 - loss: 0.6167 - val_acc: 0.6550 - val_loss: 0.6118
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 252ms/step - acc: 0.6935 - loss: 0.5702 - val_acc: 0.6690 - val_loss: 0.6304
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 17s 263ms/step - acc: 0.7070 - loss: 0.5662 - val_acc: 0.6800 - val_loss: 0.5930
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 17s 268ms/step - acc: 0.7440 - loss: 0.5098 - val_acc: 0.6100 - val_loss: 0.6868
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 17s 269ms/

### Pre-trained CNN model

In [22]:
from keras.applications import VGG16

In [23]:
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
conv_base.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 150, 150, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 150, 150, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 75, 75, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 75, 75, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 37, 37, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 37, 37, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 18, 18, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 18, 18, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 9, 9, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
conv_base.trainable = False

### Adding a densely connected classifier on top

In [25]:
input_tensor = Input(shape=(150, 150, 3))
x1 = conv_base(input_tensor)
x2 = layers.Flatten()(x1)
x3 = layers.Dense(256, activation='relu')(x2)
output_tensor = layers.Dense(2, activation='softmax')(x3)
clf2 = models.Model(input_tensor, output_tensor)

In [26]:
clf2.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 4, 4, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,812,610 (64.14 MB)

 Trainable params: 2,097,922 (8.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

### Training the new model

In [27]:
clf2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
clf2.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test));

Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - acc: 0.7990 - loss: 0.4980 - val_acc: 0.8700 - val_loss: 0.2814
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - acc: 0.9215 - loss: 0.2010 - val_acc: 0.8310 - val_loss: 0.3671
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - acc: 0.9435 - loss: 0.1505 - val_acc: 0.8470 - val_loss: 0.3750
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - acc: 0.9695 - loss: 0.0905 - val_acc: 0.8850 - val_loss: 0.2692
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 112s 2s/step - acc: 0.9910 - loss: 0.0486 - val_acc: 0.8850 - val_loss: 0.3091


### Removing the data

In [ ]:
for d in os.listdir('data/'):
    for f in os.listdir('data/' + d):
        os.remove('data/' + d + '/' + f)
    os.rmdir('data/' + d)
os.rmdir('data')